# Task - Named Entity Recognition (NER)

# Introduction

NER (Named Entity Recognition) is a fundamental task in Natural Language Processing (NLP). It involves identifying and categorizing named entities in text, like people, organizations, and locations. The main goal of NER is to assess a model's ability to classify named entities in unstructured text into predefined categories.

In this tutorial, we will evaluate the Named Entity Recognition (NER) capabilities of the GPT-3.5 Turbo language model. The tutorial uses the CoNLL 2003 dataset, a benchmark for NER tasks, to assess the model's ability to identify and classify named entities such as people, organizations, and locations in text. The notebook includes code to load the dataset, interact with the GPT-3.5 Turbo model via the OpenAI API, and evaluate the model's performance by comparing its predictions to the ground truth annotations in the CoNLL dataset.


 The CoNLL dataset consists of text data with annotated named entities: Person, Organization, Location, and Miscellaneous.

For additional information about the CoNLL benchmark: https://arxiv.org/pdf/cs/0306050v1


# Step 1 - Install Pre-requisites

In step 1, we will load the pre-requisites

We need to install the following libraries:
- `openai`: For interacting with the OpenAI API to query the LLM.
- `python-dotenv`: To manage API keys securely using environment variables.
- `datasets`: The datasets library provides easy access to a wide variety of datasets commonly used for natural language processing tasks.
- `tqdm`: Adds progress bars to loops, making it easier to monitor & visualize the progress
- `rich`: A library to render rich text (for display purposes)
- `scikit-learn`: A ML library with implmenetations of algorithms, metrics for classification, regression, clustering tasks.

In [ ]:
%pip install openai==0.28 python-dotenv datasets tqdm rich scikit-learn

Next, we will import the necessary libraries that will be used for various activities such as data processing, API interaction, and environment management tasks.

In [ ]:
import os
import openai
import random
from dotenv import load_dotenv
from rich import print as rprint
from datasets import load_dataset
from sklearn.metrics import precision_recall_fscore_support
from IPython.display import display, HTML

# Step 2 - Load LLM

Next, to access the GPT3.5 Turbo model via API, we establish a connection using an API key.


In [ ]:
# Load API key from environment file
load_dotenv(dotenv_path="../apikey.env.txt")  # replace the "file path" with the location of your API key file

APIKEY = os.getenv("APIKEY")

openai.api_key = APIKEY


A function to interact with the GPT3.5 turbo model and extract NER tokens from the model

In [ ]:
def GetModelResponse(system_content, user_content):
    system = {'role': 'system', 'content': system_content}
    user = {'role': 'user', 'content': user_content}


    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=1.0,
        messages=[system, user],
        max_tokens=2000
    )

    content = response.choices[0].message.content
    return content

# Step 3 - Load test dataset

The CoNLL dataset consists of articles with annotated entities like person, organization, location and miscellaneous.

Next, we will download and load the CoNLL 2003 dataset.

In [ ]:
# Download the CoNLL dataset
conll_data = load_dataset("conll2003")

**Exploring the CoNLL dataset**

The CoNLL dataset consists of three parts: a training set, a validation set, and a test set.

We will use the test dataset to perform NER evaluations.

In [ ]:
# Display basic statistics about the dataset
train_data = conll_data['train']
test_data = conll_data['test']
valid_data = conll_data['validation']

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(valid_data)}")
print(f"Test set size: {len(test_data)}")

**The test set consists of 3453 instances**.

As a tester, you have the option to select the number of instances (out of 3453) on which the LLM will be evaluated. **If no options are provided, the script will default to evaluating 25 instances**.

In [ ]:
# Ask user for the number of test instances to evaluate
num_test_instances = input("Enter the number of test instances to evaluate (default 25): ")
num_test_instances = int(num_test_instances) if num_test_instances else 25

# Select the random test instances
random.seed(12)
test_instances = random.sample(list(test_data), num_test_instances)


rprint(f"[blue]Number of test instances: {len(test_instances)}[/blue]")

# Step 4 - Prompt Construction

We will construct the prompt to instruct the model to identify and classify entities from the input text. Then, we will evaluate the model performance by comparing the predicted label with the ground truth

In [ ]:
# Map numeric tags to string labels for CoNLL dataset
label_mapping = {
    0: 'O',
    1: 'B-PER',
    2: 'I-PER',
    3: 'B-LOC',
    4: 'I-LOC',
    5: 'B-ORG',
    6: 'I-ORG',
    7: 'B-MISC',
    8: 'I-MISC'
}


# Function to extract NER from the model
def get_ner(tokens):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant proficient in Named entity Recogintion tasks. The goal is to identify and classify named entities (such as persons, locations, organizations, and miscellaneous entities) in a given list of tokens. The output should be a dictionary where the keys are the tokens and the values are the corresponding entity types, using the following entity tag set: ['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']."},
            {"role": "user", "content": f" For example, given the tokens ['John', 'Smith', 'lives,' 'in', 'New', 'York', 'City'], the output should be: {{'John': 'B-PER', 'Smith': 'I-PER', 'lives': 'O', 'in': 'O', 'New': 'B-LOC', 'York': 'I-LOC', 'City': 'I-LOC'}}. Now, please provide the named entity tags for the following list of tokens: {tokens}"}
        ]
    )
    ner_results = response.choices[0].message['content']

    try:
        ner_dict = eval(ner_results)
        if isinstance(ner_dict, dict):
            return ner_dict
        else:
            return {}
    except ValueError:
        return {}


Next, we'll evaluate the LLM's performance by comparing its predicted NER tags against the ground truth labels.

Correctly predicted tokens will be highlighted in green, while mismatches will be displayed in orange.

In [ ]:
# Compare LLM predictions with ground truth
predictions = []
ground_truths = []

for instance in test_instances:
    tokens = instance['tokens']
    ground_truth = {tokens[i]: label_mapping[instance['ner_tags'][i]] for i in range(len(tokens))}
    prediction = get_ner(tokens)

    print(tokens)
    print(ground_truth)
    print(prediction)

    predictions.append(prediction)
    ground_truths.append(ground_truth)

    display_text = []
    for token in tokens:
        pred_tag = prediction.get(token, 'O')
        gt_tag = ground_truth[token]
        if pred_tag == gt_tag:
            display_text.append(f"<span style='color: green;'>{token}</span>")
        else:
            display_text.append(f"<span style='color: orange;'>{token}</span>")

    display(HTML(f"<b>Input Text:</b> {' '.join(display_text)}"))
    display(HTML(f"<b>Ground Truth:</b> {ground_truth}"))
    display(HTML(f"<b>Prediction:</b> {prediction}"))
    display(HTML("<hr>"))

# Function to convert NER tags to a comparable format
def convert_to_comparable_format(ner_dict):
    return [(word, tag) for word, tag in ner_dict.items()]

# Calculate precision, recall, and F1-score
y_true = []
y_pred = []

for gt, pred in zip(ground_truths, predictions):
    gt_converted = convert_to_comparable_format(gt)
    pred_converted = convert_to_comparable_format(pred)

    for word, tag in gt_converted:
        y_true.append(tag)
        y_pred.append(pred.get(word, 'O'))  # 'O' for non-entity

precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")